In [1]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
# import spicy
import warnings
warnings.filterwarnings('ignore')
from collections import Counter
from imblearn.over_sampling import SMOTE
from collections import Counter
from imblearn.ensemble import BalancedRandomForestClassifier
from streamz import Stream
from newsapi.newsapi_client import NewsApiClient
from datetime import date, datetime, timedelta
import os
import time
from nltk.sentiment.vader import SentimentIntensityAnalyzer

Using TensorFlow backend.


In [2]:
# Get current date and the date one month ago

num_of_weeks = 52 #in weeks

current_date = date.today()
past_date = date.today() - timedelta(weeks=num_of_weeks)

In [3]:
# Create a general function that will allow the user to input their stocks and the code will populate the dataframe with the stock info

user_input = 'AMZN'
def share_price(user_input):
  url='http://download.macrotrends.net/assets/php/stock_data_export.php?t='+ user_input
  stock_price=pd.read_csv(url,skiprows=14)
  df=pd.DataFrame(stock_price)
  return df

stock_prices = share_price(user_input).copy()
stock_prices = pd.DataFrame(stock_prices)
stock_prices.columns = ['Date','Open','High','Low','Close','Volume']
stock_prices

,Date,Open,High,Low,Close,Volume
0,1997-05-16,1.8650,1.9792,1.7083,1.7292,14700000
1,1997-05-19,1.7083,1.7708,1.6250,1.7083,6106800
2,1997-05-20,1.7292,1.7500,1.6358,1.6358,5467200
3,1997-05-21,1.6042,1.6458,1.3750,1.4275,18853200
4,1997-05-22,1.4375,1.4483,1.3125,1.3958,11776800
...,...,...,...,...,...,...
5760,2020-04-07,2017.1100,2035.7200,1997.6200,2011.6000,5082216
5761,2020-04-08,2021.0000,2044.0000,2011.1500,2043.0000,3943414
5762,2020-04-09,2044.3000,2053.0000,2017.6600,2042.7600,4612716
5763,2020-04-13,2040.0000,2180.0000,2038.0000,2168.8700,6716709


In [4]:
def ma_volume(values,window):
  weights = np.repeat(1.0, window)/window
  smas = np.convolve(values,weights,'valid')
  return smas

def ma_volume(values,window):
  weights = np.repeat(1.0, window)/window
  smas = np.convolve(values,weights,'valid')
  return smas

ma_volume(df['volume'],5)

NameError: name 'df' is not defined

In [7]:
#Create a function to calculate the Maximum rolling volume using the exponential moving average 

def ema_volume(stock_prices):
  window = 5
  weights = np.exp(np.linspace(-1.,0.,window))
  weights /= weights.sum()
  max_volume=np.convolve(stock_prices['Volume'],weights,'valid')[:len(stock_prices)]
  max_volume[:window]=max_volume[window]
  max_volume = pd.DataFrame(max_volume)
  max_volume.columns = ['max_volume']
  max_volume = max_volume['max_volume'].rolling(180).max()
  max_volume_df = pd.concat([stock_prices, max_volume], axis="columns", join="inner")
  return max_volume_df

max_volume_df = ema_volume(stock_prices).copy()
max_volume_df


,Date,Open,High,Low,Close,Volume,max_volume
0,1997-05-16,1.8650,1.9792,1.7083,1.7292,14700000,NaN
1,1997-05-19,1.7083,1.7708,1.6250,1.7083,6106800,NaN
2,1997-05-20,1.7292,1.7500,1.6358,1.6358,5467200,NaN
3,1997-05-21,1.6042,1.6458,1.3750,1.4275,18853200,NaN
4,1997-05-22,1.4375,1.4483,1.3125,1.3958,11776800,NaN
...,...,...,...,...,...,...,...
5756,2020-04-01,1932.9700,1944.9600,1893.0000,1907.7000,4086565,9.973603e+06
5757,2020-04-02,1901.6400,1927.5300,1890.0000,1918.8300,4305149,9.973603e+06
5758,2020-04-03,1911.1500,1926.3300,1889.1500,1906.5900,3609870,9.973603e+06
5759,2020-04-06,1936.0000,1998.5200,1930.0200,1997.5900,5723143,9.973603e+06


In [5]:
#Create a function to calculate the Minimum rolling volume using the exponential moving average 
def ema_volume(stock_prices):
  window = 5
  weights = np.exp(np.linspace(-1.,0.,window))
  weights /= weights.sum()
  max_volume=np.convolve(stock_prices['Volume'],weights,'valid')[:len(stock_prices)]
  max_volume[:window]=max_volume[window]
  max_volume = pd.DataFrame(max_volume)
  max_volume.columns = ['max_volume']
  max_volume = max_volume['max_volume'].rolling(180).max()
  max_volume_df = pd.concat([stock_prices, max_volume], axis="columns", join="inner")
  return max_volume_df

max_volume_df = ema_volume(stock_prices).copy()
max_volume_df
def ema_volume_2(stock_prices):
  window = 5
  weights = np.exp(np.linspace(-1.,0.,window))
  weights /= weights.sum()
  min_volume=np.convolve(stock_prices['Volume'],weights)[:len(stock_prices)]
  min_volume[:window]=min_volume[window]
  min_volume = pd.DataFrame(min_volume)
  min_volume.columns = ['min_volume']
  min_volume = min_volume['min_volume'].rolling(180).min()
  min_volume_df = pd.concat([stock_prices, min_volume], axis="columns", join="inner")
  return  min_volume_df

min_volume_df = ema_volume_2(stock_prices).copy()
min_volume_df

,Date,Open,High,Low,Close,Volume,min_volume
0,1997-05-16,1.8650,1.9792,1.7083,1.7292,14700000,NaN
1,1997-05-19,1.7083,1.7708,1.6250,1.7083,6106800,NaN
2,1997-05-20,1.7292,1.7500,1.6358,1.6358,5467200,NaN
3,1997-05-21,1.6042,1.6458,1.3750,1.4275,18853200,NaN
4,1997-05-22,1.4375,1.4483,1.3125,1.3958,11776800,NaN
...,...,...,...,...,...,...,...
5760,2020-04-07,2017.1100,2035.7200,1997.6200,2011.6000,5082216,2.087173e+06
5761,2020-04-08,2021.0000,2044.0000,2011.1500,2043.0000,3943414,2.087173e+06
5762,2020-04-09,2044.3000,2053.0000,2017.6600,2042.7600,4612716,2.087173e+06
5763,2020-04-13,2040.0000,2180.0000,2038.0000,2168.8700,6716709,2.087173e+06


In [74]:
#Concat the min and max volume into the master dataframe
#import functools
#dfs = [stock_prices,max_volume_df,min_volume_df]
#volume_df = reduce(lambda left, right: left.join(right, how='outer', on='Date'), dfs)
#volume_df = stock_prices.merge(min_volume_df,on='Date').merge(max_volume_df,on='Date')
volume_df = pd.concat([stock_prices,min_volume_df['min_volume'],max_volume_df['max_volume']],axis="columns",join="inner")
volume_df

,Date,Open,High,Low,Close,Volume,min_volume,max_volume
0,1997-05-16,1.8650,1.9792,1.7083,1.7292,14700000,NaN,NaN
1,1997-05-19,1.7083,1.7708,1.6250,1.7083,6106800,NaN,NaN
2,1997-05-20,1.7292,1.7500,1.6358,1.6358,5467200,NaN,NaN
3,1997-05-21,1.6042,1.6458,1.3750,1.4275,18853200,NaN,NaN
4,1997-05-22,1.4375,1.4483,1.3125,1.3958,11776800,NaN,NaN
...,...,...,...,...,...,...,...,...
5756,2020-04-01,1932.9700,1944.9600,1893.0000,1907.7000,4086565,2.087173e+06,9.973603e+06
5757,2020-04-02,1901.6400,1927.5300,1890.0000,1918.8300,4305149,2.087173e+06,9.973603e+06
5758,2020-04-03,1911.1500,1926.3300,1889.1500,1906.5900,3609870,2.087173e+06,9.973603e+06
5759,2020-04-06,1936.0000,1998.5200,1930.0200,1997.5900,5723143,2.087173e+06,9.973603e+06


In [75]:
# Volume Trade Signal

def volume_trade_strategy(volume_df): 
    if volume_df['Volume'] > volume_df['max_volume']:
        v_signal = 1
    elif volume_df['Volume'] < volume_df['min_volume']:  
        v_signal = -1
    else:
        v_signal = 0
    
    return v_signal

volume_df['v_signal'] = volume_df.apply(volume_trade_strategy, axis=1)
volume_df

,Date,Open,High,Low,Close,Volume,min_volume,max_volume,v_signal
0,1997-05-16,1.8650,1.9792,1.7083,1.7292,14700000,NaN,NaN,0
1,1997-05-19,1.7083,1.7708,1.6250,1.7083,6106800,NaN,NaN,0
2,1997-05-20,1.7292,1.7500,1.6358,1.6358,5467200,NaN,NaN,0
3,1997-05-21,1.6042,1.6458,1.3750,1.4275,18853200,NaN,NaN,0
4,1997-05-22,1.4375,1.4483,1.3125,1.3958,11776800,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...
5756,2020-04-01,1932.9700,1944.9600,1893.0000,1907.7000,4086565,2.087173e+06,9.973603e+06,0
5757,2020-04-02,1901.6400,1927.5300,1890.0000,1918.8300,4305149,2.087173e+06,9.973603e+06,0
5758,2020-04-03,1911.1500,1926.3300,1889.1500,1906.5900,3609870,2.087173e+06,9.973603e+06,0
5759,2020-04-06,1936.0000,1998.5200,1930.0200,1997.5900,5723143,2.087173e+06,9.973603e+06,0


In [62]:
#Share Price Signal Function

def bollinger_band_signal(stock_prices):
    
    bollinger_window = 20
    sp_bollinger_mid_band = stock_prices['Close'].rolling(window=bollinger_window).mean()
    sp_bollinger_std = stock_prices['Close'].rolling(window=20).std()
    sp_bollinger_upper_band  = sp_bollinger_mid_band + (sp_bollinger_std * 2)
    sp_bollinger_lower_band  = sp_bollinger_mid_band - (sp_bollinger_std * 2)
    sp_bollinger_long = np.where(stock_prices['Close'] < sp_bollinger_lower_band, 1.0, 0.0)
    sp_bollinger_short = np.where(stock_prices['Close'] > sp_bollinger_upper_band, -1.0, 0.0)
    stock_prices['sp_signal'] = sp_bollinger_long + sp_bollinger_short
    
    return stock_prices
  
bollinger_df = bollinger_band_signal(stock_prices).copy()
bollinger_df

,Date,Open,High,Low,Close,Volume,sp_signal
0,1997-05-16,1.8650,1.9792,1.7083,1.7292,14700000,0.0
1,1997-05-19,1.7083,1.7708,1.6250,1.7083,6106800,0.0
2,1997-05-20,1.7292,1.7500,1.6358,1.6358,5467200,0.0
3,1997-05-21,1.6042,1.6458,1.3750,1.4275,18853200,0.0
4,1997-05-22,1.4375,1.4483,1.3125,1.3958,11776800,0.0
...,...,...,...,...,...,...,...
5760,2020-04-07,2017.1100,2035.7200,1997.6200,2011.6000,5082216,0.0
5761,2020-04-08,2021.0000,2044.0000,2011.1500,2043.0000,3943414,0.0
5762,2020-04-09,2044.3000,2053.0000,2017.6600,2042.7600,4612716,0.0
5763,2020-04-13,2040.0000,2180.0000,2038.0000,2168.8700,6716709,-1.0


In [63]:
signals_df = pd.concat([bollinger_df['sp_signal'], volume_df['v_signal']], axis = 'columns', join = 'inner')

In [64]:
signals_df.tail()

,sp_signal,v_signal
5756,0.0,-1
5757,0.0,-1
5758,0.0,-1
5759,0.0,0
5760,0.0,0
